This notebook is part of the *orix* documentation https://orix.readthedocs.io. Links to the documentation won’t work from the notebook.

# Sampling unit vectors from $S_2$

$S_2$ is the surface of the unit sphere, which is equivalent to the set of all unit vectors or directions in 3D space.
There are multiple use cases for discretizing and sampling this space, for example for creating custom legends for inverse pole figure maps or for creating a list of unique crystal directions.
Except for the platonic solids, it is not possible to sample the sphere surface such that each vector has an identical neighborhood, i.e. equal angles between nearest neighbors.
A number of sampling or "meshing" techniques exist, a number of which have been implemented in orix.

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from orix import plot
from orix.crystal_map import Phase
from orix.quaternion import Orientation, symmetry
from orix.sampling import S2_sampling
from orix.vector import Vector3d

## Sampling methods
Currently 7 sampling methods are supported, demonstrated below by plotting in stereographic projection, each with different regions of higher and lower sampling density.
The user should mainly decide on the `resolution` parameter, which determines the maximum angle (in degrees) between nearest neighbors except in the `random` sample where it determines the **average** angle between nearest neighbors.

In [ ]:
fig, axes = plt.subplots(2, 4, subplot_kw=dict(projection="stereographic"), 
                         figsize=(20, 10), facecolor="white")
axes = axes.ravel()

resolution = 5

for i, (name, sampling_method) in enumerate(S2_sampling.SAMPLING_METHODS.items()):
    vectors = sampling_method(resolution = 5)
    axes[i].set_title(name, fontsize=20)
    axes[i].scatter(vectors, s=2);

## Sampling the fundamental zone
Sampling of directions is most relevant when combined with crystal symmetry functionality.
In this case, we would want to sample vectors in the real crystallographic basis.
Usually one only wants to sample the fundamental zone, not the entire sphere, as in the context of a crystal, many vectors are symmetrically equivalent.
The fundamental zone of a point group represents the smallest region of directions unique by symmetry.
The higher the symmetry of the point group, the smaller the fundamental zone.

The symmetry operations are independent of the S2 sampling methods, so any set of vectors can be limited to the fundamental zone.
However, different meshes may be most appropriate for different point groups, as we illustrate.
In general, to sample vectors for crystals with $\alpha=\beta=\gamma=90^\circ$, one of the `cube` meshes is recommended.
For trigonal or hexagonal crystals that have a 3 or 6-fold axis along [001], a hexagonal mesh is recommended.
This ensures that the boundary of the fundamental zone is also sampled at regular intervals, otherwise some of these edges may appear "ragged".
For lower symmetry crystals, most samplings will be fine, and the icosahedral sampling may be recommended as points are most evenly spread.

Below we demonstrate a sampling of the fundamental zones for all 32 point groups.

In [ ]:
point_groups = symmetry._groups

color_map = {
    "triclinic": "red",
    "monoclinic": "orange",
    "orthorhombic": "green",
    "tetragonal": "blue",
    "cubic": "brown",
    "trigonal": "magenta",
    "hexagonal": "purple",
}

mesh_sampling_map = {
    "triclinic": "icosahedral",
    "monoclinic": "icosahedral",
    "orthorhombic": "spherified_cube_edge",
    "tetragonal": "spherified_cube_edge",
    "cubic": "spherified_cube_edge",
    "trigonal": "hexagonal",
    "hexagonal": "hexagonal",
}

fig, axes = plt.subplots(len(point_groups), 2, subplot_kw=dict(projection="stereographic"), figsize=(10, 5 * len(point_groups)), facecolor="white")

for i, point_group in enumerate(point_groups):
    system = point_group.system
    color = color_map[system]
    meshing_method = mesh_sampling_map[system]
    mesh = S2_sampling.SAMPLING_METHODS[meshing_method](resolution=3)
    fundamental = mesh[mesh <= point_group.fundamental_sector]
    axes[i][0].set_title(f"{point_group.name} ({system}) (upper)", fontsize=20)
    axes[i][0].scatter(fundamental, s=2, c=color)
    axes[i][1].set_title(f"{point_group.name} ({system}) (lower)", fontsize=20)
    axes[i][1].hemisphere = "lower"
    axes[i][1].scatter(fundamental, s=2, c=color);